In [1]:
import folium
from folium import Map, PolyLine, Marker
import json


In [2]:
# Original data in -180/+180 format with dateline crossing issues
ship_track_180 = [
    [170, 30],   # 170°E
    [175, 31],   # 175°E  
    [179, 32],   # 179°E
    [-179, 33],  # 179°W (dateline jump!)
    [-175, 34],  # 175°W
    [-170, 35]   # 170°W
]

storm_track_180 = [
    [178, 25],   # 178°E
    [180, 26],   # 180° (at dateline)
    [-178, 27],  # 178°W (dateline jump!)
    [-172, 28],  # 172°W
    [-165, 29]   # 165°W
]

print("Original -180/+180 data with dateline crossing problems:")
print("Ship track (-180/+180):")
for i, (lon, lat) in enumerate(ship_track_180):
    print(f"  Point {i+1}: {lon}°, {lat}°N")

print("\nStorm track (-180/+180):")
for i, (lon, lat) in enumerate(storm_track_180):
    print(f"  Point {i+1}: {lon}°, {lat}°N")
    
print("\n❌ Problem: Notice the sudden jump from +179° to -179° - this breaks visualization!")


Original -180/+180 data with dateline crossing problems:
Ship track (-180/+180):
  Point 1: 170°, 30°N
  Point 2: 175°, 31°N
  Point 3: 179°, 32°N
  Point 4: -179°, 33°N
  Point 5: -175°, 34°N
  Point 6: -170°, 35°N

Storm track (-180/+180):
  Point 1: 178°, 25°N
  Point 2: 180°, 26°N
  Point 3: -178°, 27°N
  Point 4: -172°, 28°N
  Point 5: -165°, 29°N

❌ Problem: Notice the sudden jump from +179° to -179° - this breaks visualization!


In [3]:
def transform_to_360(track):
    """Transform longitude coordinates from -180/+180 to 0-360 system."""
    transformed = []
    for lon, lat in track:
        # Convert negative longitudes to 0-360 range
        if lon < 0:
            lon_360 = lon + 360
        else:
            lon_360 = lon
        transformed.append([lon_360, lat])
    return transformed

# Transform both tracks to 0-360° system
ship_track_360 = transform_to_360(ship_track_180)
storm_track_360 = transform_to_360(storm_track_180)

print("Transformed to 0-360° system:")
print("Ship track (0-360°):")
for i, (lon, lat) in enumerate(ship_track_360):
    print(f"  Point {i+1}: {lon}°E, {lat}°N")

print("\nStorm track (0-360°):")
for i, (lon, lat) in enumerate(storm_track_360):
    print(f"  Point {i+1}: {lon}°E, {lat}°N")
    
print("\n✅ Solution: No more jumps! Smooth progression: 179° → 181° → 185° → 190°")


Transformed to 0-360° system:
Ship track (0-360°):
  Point 1: 170°E, 30°N
  Point 2: 175°E, 31°N
  Point 3: 179°E, 32°N
  Point 4: 181°E, 33°N
  Point 5: 185°E, 34°N
  Point 6: 190°E, 35°N

Storm track (0-360°):
  Point 1: 178°E, 25°N
  Point 2: 180°E, 26°N
  Point 3: 182°E, 27°N
  Point 4: 188°E, 28°N
  Point 5: 195°E, 29°N

✅ Solution: No more jumps! Smooth progression: 179° → 181° → 185° → 190°


In [4]:
# Create map centered at the dateline (180°)
m = folium.Map(location=[30, 180], zoom_start=3, tiles='CartoDB positron')

# Add ship track using 0-360° coordinates directly
ship_locations = [[lat, lon] for lon, lat in ship_track_360]  # Folium uses [lat, lon]
folium.PolyLine(
    locations=ship_locations,
    color='blue',
    weight=4,
    opacity=0.8,
    tooltip='Ship Track (0-360° coordinates)'
).add_to(m)

# Add storm track using 0-360° coordinates directly
storm_locations = [[lat, lon] for lon, lat in storm_track_360]  # Folium uses [lat, lon]
folium.PolyLine(
    locations=storm_locations,
    color='red',
    weight=4,
    opacity=0.8,
    tooltip='Storm Track (0-360° coordinates)'
).add_to(m)

# Add markers showing the actual coordinates being used
for i, (lon, lat) in enumerate(ship_track_360):
    folium.Marker(
        [lat, lon],
        popup=f'Ship Point {i+1}: {lon}°E, {lat}°N',
        tooltip=f'Ship {i+1}',
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

for i, (lon, lat) in enumerate(storm_track_360):
    folium.Marker(
        [lat, lon],
        popup=f'Storm Point {i+1}: {lon}°E, {lat}°N',
        tooltip=f'Storm {i+1}',
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# Add dateline reference
folium.PolyLine(
    locations=[[-90, 180], [90, 180]],
    color='black',
    weight=2,
    opacity=0.5,
    dash_array='5,5',
    tooltip='International Date Line (180°)'
).add_to(m)

# Display the map
m


In [5]:
# Create comparison map showing both approaches
m_compare = folium.Map(location=[30, 180], zoom_start=3, tiles='CartoDB positron')

# WRONG: Plot using original -180/+180 data (will show wrapping issues)
ship_locations_wrong = [[lat, lon] for lon, lat in ship_track_180]
folium.PolyLine(
    locations=ship_locations_wrong,
    color='orange',
    weight=6,
    opacity=0.7,
    tooltip='❌ WRONG: -180/+180 data (wraps around globe)',
    dash_array='10,5'
).add_to(m_compare)

# RIGHT: Plot using 0-360° data (smooth crossing)
ship_locations_right = [[lat, lon] for lon, lat in ship_track_360]
folium.PolyLine(
    locations=ship_locations_right,
    color='blue',
    weight=4,
    opacity=0.9,
    tooltip='✅ RIGHT: 0-360° data (smooth crossing)'
).add_to(m_compare)

# Add dateline reference
folium.PolyLine(
    locations=[[-90, 180], [90, 180]],
    color='black',
    weight=2,
    opacity=0.5,
    dash_array='5,5',
    tooltip='International Date Line (180°)'
).add_to(m_compare)

# Add legend
legend_html = '''
<div style="position: fixed; 
            top: 10px; right: 10px; width: 200px; height: 90px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px">
<b>Dateline Crossing Comparison</b><br>
<i class="fa fa-minus" style="color:orange"></i> Wrong: -180/+180 (wraps)<br>
<i class="fa fa-minus" style="color:blue"></i> Right: 0-360° (smooth)
</div>
'''
m_compare.get_root().html.add_child(folium.Element(legend_html))

m_compare


In [6]:
# Save the corrected map
m.save('dateline_demo_corrected.html')
print("✅ Corrected map saved as 'dateline_demo_corrected.html'")
print("\nKey takeaways:")
print("1. Start with -180/+180 data that has dateline crossing issues")
print("2. Transform to 0-360° system to eliminate jumps")
print("3. Folium/Leaflet.js handles 0-360° coordinates perfectly!")
print("4. No need to convert back to -180/+180 - that's unnecessary legacy code")


✅ Corrected map saved as 'dateline_demo_corrected.html'

Key takeaways:
1. Start with -180/+180 data that has dateline crossing issues
2. Transform to 0-360° system to eliminate jumps
3. Folium/Leaflet.js handles 0-360° coordinates perfectly!
4. No need to convert back to -180/+180 - that's unnecessary legacy code
